In [1]:
import requests
from lxml import etree
import pandas as pd

#请求目标网站并解析
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36"}
url = "http://car.bitauto.com/xuanchegongju/?l=8&mid=8"
res = requests.get(url,headers=headers).text#用get方式发送请求并获得响应
res = etree.HTML(res)

#获取爬取元素
# model_box = res.xpath("//div[@class='search-result-list-item']")  得到的是一个list，下面要直接再次路径上使用相对路径的话，需要for循环一个一个读取
model_name_list = res.xpath("//div[@class='search-result-list-item']/a/p[@class='cx-name text-hover']/text()")
model_price_list = res.xpath("//div[@class='search-result-list-item']/a/p[@class='cx-price']/text()")
model_pig_list = res.xpath("//div[@class='search-result-list-item']/a/img[@class='img']/@src")

#多个元素list直接合并创建DataFrame
car_info = pd.DataFrame(data = {'name':model_name_list,'price':model_price_list,'pig_url':model_pig_list})


In [2]:
#拆分价格得到新的DF
df2 = car_info['price'].str.split('-',expand = True)
#获取最低最高价格
car_info['最低价格'] = df2.iloc[:,0]
car_info['最高价格'] = df2.iloc[:,1].str.extract(r'(\d+\.?\d*)')  #正则表达式
car_info.drop('price',axis = 1,inplace = True)   #删除价格列
#pig_url处理
car_info['pig_url'] = ['https: %s' % i for i in car_info['pig_url']]
car_info

,name,pig_url,最低价格,最高价格
0,探岳,https: //img2.bitautoimg.com/autoalbum/files/2...,18.59,31.39
1,途岳,https: //img2.bitautoimg.com/autoalbum/files/2...,16.58,22.38
2,途观L,https: //img4.bitautoimg.com/autoalbum/files/2...,21.88,31.18
3,途昂,https: //img3.bitautoimg.com/autoalbum/files/2...,29.99,50.59
4,途铠,https: //img2.bitautoimg.com/autoalbum/files/2...,11.49,15.99
5,探歌,https: //img4.bitautoimg.com/autoalbum/files/2...,13.58,20.38
6,途观,https: //img1.bitautoimg.com/autoalbum/files/2...,19.48,23.18
7,探影,https: //img1.bitautoimg.com/autoalbum/files/2...,11.49,15.99
8,探岳X,https: //img1.bitautoimg.com/autoalbum/files/2...,23.58,29.58
9,途昂X,https: //img1.bitautoimg.com/autoalbum/files/2...,28.99,43.99


In [41]:
#保存数据
car_info.to_csv('./vw_suv.csv')